# *Skodge Truhlar KIE correction*
This is an IPython Notebook interface to calculate tunnelling corrections via the Skodge Truhlar equations (*J. Phys. Chem.* **1981**, 85, 624-626.) using the *PyQuiver* and *cctk* packages. The code below will guide you through using *PyQuiver* and *cctk* through a native Python interface. The same steps could be reproduced in the Python interpreter or in a `.py` file.

The initial KIE calculation requires hessians for the transition state and the starting material. The Skodge Truhlar equation additionally requires the energies of each SM/PR/TS (single point electronic energies). For Gaussian calculations this will require only the output files for the transition state and pre-transition state calculations. For Orca calculations the .hess and .out files are necessary

### Setup
Import necessary package elements.
sys.path.append should point to the src directory of PyQuiver. Note: Paths for all operating systems (including Windows) should use '/'

cctk can be installed via "pip install cctk" or from github (ekwan/cctk)

In [ ]:
import numpy as np
import sys
sys.path.append("../src")
from kie import KIE_Calculation
import cctk

"""Constants are imported from PHYSICAL_CONSTANTS (PyQuiver/src)

Constants:
    h: Planck's constant in J * s
    c: speed of light in cm / s
    kB: Boltzmann's constant in J / K
    Eh: energy of a hartree in J
"""
from constants import PHYSICAL_CONSTANTS
h  = PHYSICAL_CONSTANTS["h"]
c  = PHYSICAL_CONSTANTS["c"]
kB = PHYSICAL_CONSTANTS["kB"]
Eh = PHYSICAL_CONSTANTS["Eh"]

### Extracting SPE energy from electronic structure calculations

Electronic structure output files are read by cctk depending on the value of the *software* variable. The final molecule is chosen, and its energy is returned.

In [ ]:
def get_energy(software,path):
    """Function to get energy from electronic structure calculation
    Args:
        software (string): the electronic structure package used. supported values: `orca` or `gauss`
        path (string): relative or exact path to output file 
    
    Returns:
        float: energy in Eh

    Raises:
        Exception: if software value passed is not as expected
    """
    if software == 'orca':
        ensemble = cctk.OrcaFile.read_file(path).ensemble
    elif software == 'gauss':
        ensemble = cctk.GaussianFile.read_file(path).ensemble
    else:
        raise Exception("Check software variable passed")
    molecule = ensemble.molecules[-1]
    properties_dict = ensemble.get_properties_dict(molecule)
    return properties_dict["energy"]

### The Skodge Truhlar equation

The Skodge Truhlar equation is an alternative approximation for calculating transmission coefficients. (*J. Phys. Chem.* **1981**, 85, 624-626.) This strategy might be more applicable in cases of very large frequencies.

The the natural vibrational frequency is calculated from the reduced mass ($\mu$) and the force constant (k) of the imaginary mode. This is handled automatically in PyQuiver.
$$ \nu^\ddag = {1 \over{2 \pi}} * \sqrt{k^\ddag \over \mu^\ddag} $$

The ST equation is definited by two key terms $\alpha$ and $\beta$, where $ \alpha = {{2\pi}\over{h\nu^\ddag}} $ and $ \beta = {1\over{k_BT}}$ as well as the barrier height (electronic energy) in the exothermic direction ($V = V_{TS} - max(V_{SM},V_{PR})$):



The ST equation has two forms depending on the vibrational frequency and temperature.

In cases of a relatively small $\nu^\ddag$ and high T ($\alpha > \beta$):
$$ \kappa = {{\beta\pi/\alpha}\over{sin(\beta\pi/\alpha)}} - {\beta\over{\alpha - \beta}}e^{(\beta-\alpha)V} $$

Conversely, in cases of a relatively large $\nu^\ddag$ and low T ($\alpha < \beta$):

$$ \kappa = {\beta\over{\beta-\alpha}}(e^{(\beta-\alpha)V}-1) $$

In states where $\alpha = \beta$ :
$$ \kappa = {{\beta\pi/\alpha}\over{sin(\beta\pi/\alpha)}}$$

The tunnelling correction is defined as $\kappa_{light}/\kappa_{heavy}$


In [ ]:
def st_factor(imag_cm,temp_K,sm_energy_Eh,pr_energy_Eh,ts_energy_Eh):
    """This function calculates the Skodge Truhlar transmission coefficient
    Args:
        imag_cm (float): natural vibrational frequency of the transition state mode in 1 / cm
        temp_K (int): temperature of reaction in K 
        sm_energy_Eh (float): energy of starting structure in hartree
        pr_energy_Eh (float): energy of product structure in hartree
        ts_energy_Eh (float): energy of transition state in hartree

    Returns:
        float: Skodge Truhlar transmission coefficient
    """

    #: natural vibrational frequency is converted to hertz with the speed of light and made positive
    imag_hz = np.abs(imag_cm * c)

    #: sm/pr/ts energies are converted to J
    pr_energy_J = pr_energy_Eh * Eh
    ts_energy_J = ts_energy_Eh * Eh
    sm_energy_J = sm_energy_Eh * Eh

    #: barrier is calculated in the exothermic direction in J
    barr_J = ts_energy_J - max(pr_energy_J,sm_energy_J)

    #: ST parameters alpha and beta are calculated both in J**-1
    alpha = 2 * np.pi / (h * imag_hz)
    beta = 1 / (kB * temp_K)

    #: Equation used is dependent on relation of alpha and beta, both are unitless
    if alpha == beta:
        return (beta * np.pi / alpha)/(np.sin(beta * np.pi / alpha))
    elif alpha > beta:
        return (beta * np.pi / alpha)/(np.sin(beta * np.pi / alpha)) - beta/(alpha-beta)*np.exp((beta-alpha)*barr_J)
    else:
        return beta/(beta-alpha)*(np.exp((beta-alpha)*barr_J)-1)

def st_corrrection(imag_light_cm,imag_heavy_cm,temp_K,sm_energy_Eh,pr_energy_Eh,ts_energy_Eh):
    """This function uses st_factor to calculate the ratio of tunnelling coefficient for the light and heavy isotopes
    Args:
        imag_light_cm (float): light isotope natural vibrational frequency of the transition state mode in 1 / cm
        imag_heavy_cm (float): heavy isotope natural vibrational frequency of the transition state mode in 1 / cm
        temp_K (int): temperature of reaction in K 
        sm_energy_Eh (float): energy of starting structure in hartree
        pr_energy_Eh (float): energy of product structure in hartree
        ts_energy_Eh (float): energy of transition state in hartree

    Returns:
        float: Skodge Truhlar tunnelling correction factor
    """
    light_factor = st_factor(imag_light_cm,temp_K,sm_energy_Eh,pr_energy_Eh,ts_energy_Eh)
    heavy_factor = st_factor(imag_heavy_cm,temp_K,sm_energy_Eh,pr_energy_Eh,ts_energy_Eh)
    return light_factor/heavy_factor

### PyQuiver implementation
The following function uses the above function for the ST tunnelling correction to a PyQuiver generated list of KIEs.

In [ ]:
def calc_frequency(kie,kiecalc):
        """This function uses PyQuiver to calculate the natural vibrational frequencies
        Args:
            kie (KIE): isotopologue to calculate frequency of
            kiecalc (KIE_CALCULATION): kie calculation to extract configuration parameters from

        Returns:
            light_imag_freq (float), heavy_imag_freq (float): light and heavy imaginary frequencies in 1 / cm
        """

        light_imag_freqs = kie.ts_tuple[0].calculate_frequencies(kiecalc.config.imag_threshold, scaling=kiecalc.config.scaling)[1]
        heavy_imag_freqs = kie.ts_tuple[1].calculate_frequencies(kiecalc.config.imag_threshold, scaling=kiecalc.config.scaling)[1]
        return min(light_imag_freqs), min(heavy_imag_freqs)

def st_KIE(kiecalc,sm_energy_Eh,pr_energy_Eh,ts_energy_Eh):
    """This function calculates Skodge Truhlar corrected KIEs from a KIE_CALCULATION
    Args:
        kiecalc (KIE_Calculation): KIE Calculation
        sm_energy_Eh (float): energy of starting structure in hartree
        pr_energy_Eh (float): energy of product structure in hartree
        ts_energy_Eh (float): energy of transition state in hartree

    Returns:
            dictionary{isotopologue}: dictionary of isotopologues

    Raise:
        Exception: if KIE calculation is determined to be an EIE calculation
    """

    sk_kie_dict = {}

    if kiecalc.eie_flag != 0:
        raise Exception("ST Correction not appropriate for EIE calculation")

        #: Check whether to use relative or absolute KIEs, calculate reference tunnelling correction if relative
    if kiecalc.config.reference_isotopologue != "default" and kiecalc.config.reference_isotopologue != "none":
        ref = kiecalc.KIES[kiecalc.config.reference_isotopologue]
        imag_light_cm, imag_heavy_cm = calc_frequency(ref,kiecalc)
        ref_tunnel_corr = st_corrrection(imag_light_cm,imag_heavy_cm,kiecalc.config.temperature,sm_energy_Eh,pr_energy_Eh,ts_energy_Eh)
        sk_kie_dict[kiecalc.config.reference_isotopologue] = ref.value[0] * ref_tunnel_corr
    else:
        ref_tunnel_corr = 1

    for key, value in kiecalc.KIES.items():
        if key != kiecalc.config.reference_isotopologue:
            #: extract light and heavy imaginary frequencies from pyquiver
            imag_light_cm, imag_heavy_cm = calc_frequency(value,kiecalc)
            
            #: calculate tunnelling correction and print result
            tuncorrection = st_corrrection(imag_light_cm,imag_heavy_cm,kiecalc.config.temperature,sm_energy_Eh,pr_energy_Eh,ts_energy_Eh) / ref_tunnel_corr
            sk_kie_dict[key] = value.value[0] * tuncorrection

    return sk_kie_dict

def print_st(kiecalc,sm_energy_Eh,pr_energy_Eh,ts_energy_Eh):
    """This function prints Skodge Truhlar corrected KIEs from a KIE_CALCULATION
    Args:
        kiecalc (KIE_Calculation): KIE Calculation
        sm_energy_Eh (float): energy of starting structure in hartree
        pr_energy_Eh (float): energy of product structure in hartree
        ts_energy_Eh (float): energy of transition state in hartree
    """
    
    print("Skodge Truhlar Corrected Absolute KIE:")
    print("Isotopologue                                              uncorrected   Skodge Truhlar")
    print("                                                               KIE           KIE")

    sk_kie_dict = st_KIE(kiecalc,sm_energy_Eh,pr_energy_Eh,ts_energy_Eh)

    #: Iterate through Isotopologues to print uncorrected and ST corrected KIEs
    for key, value in kiecalc.KIES.items():
        if key != kiecalc.config.reference_isotopologue:
            #: extract light and heavy imaginary frequencies from pyquiver
            
            spaces = " " * max((10 - len(key)),1)
            print("isotopologue:" + spaces + key, end ="                                      ")
            print("%0.4f" % value.value[0], end ="        ")
            print("%0.4f" % sk_kie_dict[key])
            
    #: display reference information
    if kiecalc.config.reference_isotopologue != "default" and kiecalc.config.reference_isotopologue != "none":
        print()
        print("KIEs referenced to isotopologue "+ kiecalc.config.reference_isotopologue +", whose absolute KIEs are:")
        spaces = " " * max((10 - len(kiecalc.config.reference_isotopologue)),1)
        print("isotopologue:" + spaces + kiecalc.config.reference_isotopologue, end ="                                      ")
        print("%0.4f" % kiecalc.KIES[kiecalc.config.reference_isotopologue].value[0], end ="        ")
        print("%0.4f" % sk_kie_dict[kiecalc.config.reference_isotopologue])


### Running calculation
With the necessary functions defined, the correction can be run on electronic structure data. Here the paths reference the tutorial data for the Claisen reaction. Note that for this reaction the ST corrected KIEs nearly exactly match the inverted parabola values.

First the software is set which allows cctk to appropriately extract the energies. Then, the path to the *PyQuiver* configuration file is defined (see tutorial for details). Finally, relevent electronic structure calculations are set. For generality the extension is ommitted and added as needed. Alternatively, these paths can of course be manually set below when calling the appropriate function.

The first example is for Orca and the second for Gaussian:

In [ ]:
# set computational software
# gauss for Gaussian
# orca for Orca
software = "orca"

#location of pyquiver configuration file, see documentation/tutorial for details
pyquivconf = "../tutorial/orca/claisen_demo.config"

#include basename not extension
smpath = "../tutorial/orca/claisen_gs_freq"
prpath = "../tutorial/orca/claisen_gs_freq"
tspath = "../tutorial/orca/claisen_ts_freq"

kiecalc = KIE_Calculation(pyquivconf, smpath + ".hess", tspath + ".hess", style="orca")
print(kiecalc)
print()
print_st(kiecalc,get_energy(software,smpath+".out"),get_energy(software,prpath+".out"),get_energy(software,tspath+".out"))

It is also possible to directly extract the KIEs (in addition to printing them) as a dictionary of isotopologues (relative to reference isotopologue, which is recorded in the dictionary as an absolute KIE)

Here we extract and print the ST corrected C4 KIE:

In [ ]:
st = st_KIE(kiecalc,get_energy(software,smpath+".out"),get_energy(software,prpath+".out"),get_energy(software,tspath+".out"))
print("%0.4f" % st["C4"])

Here is the same calculation as above, except using Gaussian output files

In [ ]:
# set computational software
# gauss for Gaussian
# orca for Orca
software = "gauss"

#location of pyquiver configuration file, see documentation/tutorial for details
pyquivconf = "../tutorial/gaussian/claisen_demo.config"

#In this case the paths can include the extension since pyquiver and gaussian both use the gaussian output files
smpath = "../tutorial/gaussian/claisen_gs.out"
prpath = "../tutorial/gaussian/claisen_gs.out"
tspath = "../tutorial/gaussian/claisen_ts.out"

kiecalc = KIE_Calculation(pyquivconf, smpath, tspath)
print(kiecalc)
print()
print_st(kiecalc,get_energy(software,smpath),get_energy(software,prpath),get_energy(software,tspath))